In [1]:
import os

In [2]:
os.chdir('../')

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class TrainingConfig:
    root_dir: str
    trained_model_path: str
    learning_rate: float
    weight_decay: float
    num_epochs: int
    patience: int
    num_layers_to_unfreeze: int
    pretrained_model_path: str



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [15]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
# from cnnClassifier.entity import DataPreprocessingConfig
from cnnClassifier.entity.config_entity import PrepareBaseModelConfig
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_cfg = self.config.training
        pretrained_model_path = self.config.prepare_base_model.updated_base_model_path
        create_directories([training_cfg.root_dir])
        return TrainingConfig(
            root_dir=training_cfg.root_dir,
            trained_model_path=training_cfg.trained_model_path,
            pretrained_model_path=pretrained_model_path,
            learning_rate=0.0001,          # you can push this to yaml later
            weight_decay=1e-4,
            num_epochs=5,
            patience=5,
            num_layers_to_unfreeze=30
        )
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            model_name=self.params.MODEL_NAME,
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

In [16]:
import os
import time
import torch
from torch.utils.tensorboard import SummaryWriter



class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        self.writer = None
        self.ckpt_path = self.config.checkpoint_model_filepath

    def _create_tb_writer(self):
        """Create a TensorBoard writer with timestamped log dir"""
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        os.makedirs(tb_running_log_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=tb_running_log_dir)
        print(f"[TensorBoard] Logging at: {tb_running_log_dir}")
        return self.writer

    def _save_checkpoint(self, model, optimizer, epoch, val_loss):
        """Save model checkpoint if it's the best so far"""
        os.makedirs(os.path.dirname(self.ckpt_path), exist_ok=True)

        checkpoint = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_loss": val_loss,
        }
        torch.save(checkpoint, self.ckpt_path)
        print(f"[Checkpoint] Model saved at {self.ckpt_path}")

    def get_tb_writer(self):
        """Public method to get TensorBoard writer"""
        if self.writer is None:
            self._create_tb_writer()
        return self.writer


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

class TrainModel:
    def __init__(self, config: TrainingConfig, model: nn.Module,
                 train_loader, val_loader,callbacks=None, device="cuda"):
        self.config = config
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.callbacks = callbacks if callbacks else []
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load pretrained (updated) weights if available
        if self.config.pretrained_model_path and Path(self.config.pretrained_model_path).exists():
            print(f"📥 Loading pretrained weights from {self.config.pretrained_model_path}")
            self.model.load_state_dict(torch.load(self.config.pretrained_model_path, map_location=device))

        # Loss, optimizer, scheduler
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=self.config.learning_rate,
            weight_decay=self.config.weight_decay
        )
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        print(f"\n🚀 Starting Training for {self.config.num_epochs} epochs")
        print(f"   - Learning rate: {self.config.learning_rate}")
        print(f"   - Weight decay: {self.config.weight_decay}")
        print(f"   - Patience: {self.config.patience}")
        print(f"   - Unfreezing last {self.config.num_layers_to_unfreeze} layers")

        best_acc = 0.0
        patience_counter = 0
         # 🔹 If TensorBoard is available
        tb_writer = None
        for cb in self.callbacks:
            if hasattr(cb, "get_tb_writer"):
                tb_writer = cb.get_tb_writer()

        for epoch in range(self.config.num_epochs):
            # 🔹 Training
            self.model.train()
            running_loss, correct_train, total_train = 0.0, 0, 0

            for images, labels in self.train_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct_train += (preds == labels).sum().item()
                total_train += labels.size(0)

            train_acc = 100 * correct_train / total_train
            epoch_loss = running_loss / len(self.train_loader.dataset)
            

            # 🔹 Validation
            self.model.eval()
            correct_val, total_val = 0, 0
            with torch.no_grad():
                for images, labels in self.val_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = self.model(images)
                    _, preds = torch.max(outputs, 1)
                    correct_val += (preds == labels).sum().item()
                    total_val += labels.size(0)

            val_acc = 100 * correct_val / total_val

            print(f"📊 Epoch {epoch+1}/{self.config.num_epochs} "
                  f"| Loss: {epoch_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
            # 🔹 TensorBoard logging
            if tb_writer:
                tb_writer.add_scalar("Loss/train", epoch_loss, epoch)
                tb_writer.add_scalar("Accuracy/train", train_acc, epoch)
                tb_writer.add_scalar("Accuracy/val", val_acc, epoch)
            # 🔹 Save best model
            if val_acc > best_acc:
                best_acc = val_acc
                for cb in self.callbacks:
                    if hasattr(cb, "_save_checkpoint"):
                        cb._save_checkpoint(self.model, self.optimizer, epoch, epoch_loss)
                torch.save(self.model.state_dict(), self.config.trained_model_path)
                print(f"   💾 Best model updated @ {val_acc:.2f}% "
                      f"-> saved to {self.config.trained_model_path}")
                patience_counter = 0
            else:
                patience_counter += 1

            # 🔹 Early stopping
            if patience_counter >= self.config.patience:
                print("⏹️ Early stopping triggered")
                break

            self.scheduler.step()

        print(f"✅ Training completed! Best Val Accuracy: {best_acc:.2f}%")
        return best_acc


In [20]:
from cnnClassifier.pipeline.stage_03_data_preprocessing import DataPreprocessingTrainingPipeline
from cnnClassifier.pipeline.stage_02_prepare_base_model import PrepareBaseModelTrainingPipeline


 # 1. Load training config
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    prepare_pipeline = PrepareBaseModelTrainingPipeline()
    updated_model, model_architecture = prepare_pipeline.main()
    print(f"🚚 Model architecture ready: {model_architecture}")
    # 2. Get data from preprocessing stage
    data_pipeline = DataPreprocessingTrainingPipeline()
    train_loader, val_loader, test_loader = data_pipeline.main()
    print(f"🚚 Data loaders ready: Train({len(train_loader.dataset)}), Val({len(val_loader.dataset)}), Test({len(test_loader.dataset)})")
    
    # 3. Initialize Training component
    trainer = TrainModel(
        model=model_architecture,
        config=training_config,
        train_loader=train_loader,
        val_loader=val_loader,
        callbacks=[prepare_callbacks]
        
    )

    # 4. Train the model
    trainer.train()

except Exception as e:
    
    raise e

[2025-10-02 18:00:28,821: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-02 18:00:28,825: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-02 18:00:28,827: INFO: common: created directory at: artifacts]
[2025-10-02 18:00:28,829: INFO: common: created directory at: artifacts/training]
[2025-10-02 18:00:28,831: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-10-02 18:00:28,833: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-10-02 18:00:28,836: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-02 18:00:28,839: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-02 18:00:28,841: INFO: common: created directory at: artifacts]
[2025-10-02 18:00:28,842: INFO: common: created directory at: artifacts/prepare_base_model]
[Init] PrepareBaseModel initialized with model: efficientnet_b0
[Pipeline] Preparing base model...
[Step 1] Loadi

Traceback (most recent call last):
  File "c:\Users\el papi\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\el papi\AppData\Local\Temp\ipykernel_35528\3065248352.py", line 30, in <module>
    trainer.train()
    ~~~~~~~~~~~~~^^
  File "C:\Users\el papi\AppData\Local\Temp\ipykernel_35528\2268326157.py", line 58, in train
    running_loss += loss.item() * images.size(0)
                    ~~~~~~~~~^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\el papi\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2194, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "c:\Users\el papi\miniconda3\Lib\site-packages\IPython\core\ultratb.py", line 1179,

In [24]:
config = ConfigurationManager()
training_config = config.get_training_config()
print(TrainingConfig)

[2025-10-02 22:30:46,830: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-02 22:30:46,835: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-02 22:30:46,836: INFO: common: created directory at: artifacts]
[2025-10-02 22:30:46,838: INFO: common: created directory at: artifacts/training]
<class '__main__.TrainingConfig'>
